In [1]:
import os

In [2]:
%pwd

'd:\\Computer_Vision\\pouravi_computer_vision_project\\cotton_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig :
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    testing_data : Path
    params_epochs : int
    params_batch_size : int
    params_image_size : list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "train")
        testing_data = os.path.join(self.config.data_ingestion.unzip_dir, "test")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            testing_data = Path(testing_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [6]:
import os
import urllib.request as request
from zipfile import  ZipFile
import tensorflow as tf
import time

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self): 
         datagenerator_kwargs = dict(
                                  rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
         train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
            )   
         testgenerator_kwargs = dict(rescale = 1./255)
         test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **testgenerator_kwargs
            )
         self.train_generator = train_datagenerator.flow_from_directory(directory = self.config.training_data,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
         self.test_generator = test_datagenerator.flow_from_directory(directory = self.config.testing_data,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self,callback_list: list,):
        self.model.fit(
            self.train_generator,
            validation_data=self.test_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=len(self.train_generator),
            validation_steps=len(self.test_generator),
            callbacks=callback_list
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
         

    
    

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-05-06 15:58:08,549: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-06 15:58:08,549: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-06 15:58:08,563: INFO: common: created directory at: artifacts]
[2024-05-06 15:58:08,567: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-05-06 15:58:08,573: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-05-06 15:58:08,575: INFO: common: created directory at: artifacts\training]


Found 1951 images belonging to 4 classes.
Found 18 images belonging to 4 classes.
61/61 [==============================] - 76s 1s/step - loss: 9.7929 - accuracy: 0.6422 - val_loss: 2.7109 - val_accuracy: 0.7222
